# Upload Structure into AiiDA

In [ ]:
from __future__ import print_function

from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

from aiida.orm.querybuilder import QueryBuilder
from aiida.orm.data.structure import StructureData
from aiida.orm.calculation import Calculation

import ase.io
import numpy as np
import ipywidgets as ipw
from base64 import b64decode
from IPython.display import display, clear_output, Image
from fileupload import FileUploadWidget
from tempfile import NamedTemporaryFile

## Step 1: Upload file

In [ ]:
def on_file_upload(c):
    global traj
    with upload_out:
        clear_output()
        tmp = NamedTemporaryFile(suffix=file_upload.filename)
        f = open(tmp.name, "w")
        f.write(file_upload.data)
        f.close()
        traj = ase.io.read(tmp.name, index=":")
        tmp.close()
        #update_view()
        print("Uploaded file contained %d structures:"%len(traj))
        print(str(traj)+"\n")
        formula = traj[0].get_chemical_formula()
        inp_descr.value = "upload(%s)" % formula
        
#TODO: FileUploadWidget doesn't fire event when same file is uploaded twice
file_upload = FileUploadWidget("Upload Structure")
file_upload.observe(on_file_upload, names='data')
upload_out = ipw.Output()

display(file_upload, upload_out)

## Step 2: Store in AiiDA Database

In [ ]:
def on_click_store(b):
    global traj
    with store_out:
        clear_output()
        for atoms in traj:
            #AiiDA requires structures to have cell
            if np.all(atoms.cell == 0.0):
                atoms.center(vacuum=0.1)
            s = StructureData(ase=atoms)
            s.description = inp_descr.value
            s.store()
            print("Stored in AiiDA: "+repr(s))

inp_descr = ipw.Text(placeholder="Description (optional)")   
btn_store = ipw.Button(description='Store in AiiDA')
btn_store.on_click(on_click_store)
store_out = ipw.Output()
display(ipw.HBox([btn_store, inp_descr]), store_out)